# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [132]:
import urllib
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4)

urllib.request.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
urllib.request.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.as_matrix()
test = df_test.as_matrix()

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
Try to understand each step of the learning algorithm and comment each line.

In [90]:
def learn_reg_ERM(X,y,lbda):
    # maximum iterations of algorithm
    max_iter = 200
    # Threshold
    e  = 0.001
    # Step size/ learning rate
    alpha = 1.
    # init weights randomly
    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        # Compute intermediate values
        h = np.dot(X,w)
        # Compute hinge loss and gradient
        l,lg = loss(h, y)
        print('loss: {}'.format(np.mean(l)))
        # Compute L2 regularizer and gradient
        r,rg = reg(w, lbda)
        # Compute dot product of X and hinge loss gradient, add regularizer gradient
        # = full gradient
        g = np.dot(X.T,lg) + rg 
        # For all steps except the first
        if (k > 0):
            # Adjust learning rate
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        # Adjust weights
        w = w - alpha * g
        # End if < threshold
        if (np.linalg.norm(alpha * g) < e):
            break
        # save old gradient in order to be able to compute change
        g_old = g
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss at the points $h_i$. The partial derivative of the hinge loss $h_i$ with respect to the $i$-th position of the weight vector $\textbf{w}$ is $g_{i} = −y x_{i}$ if $l_{i} > 0$, else $g_{i} = 0$).

In [91]:
def loss(h, y):
    a = 1-h*y
    l = np.empty(h.shape)
    g = np.empty(h.shape)
    for i in range(0,a.size):
        l[i] = max(0,a[i])
        if l[i] > 0:
            g[i] = -y[i]
        else:
            g[i] = 0
    return l, g

In [82]:
X = X_train
w = np.random.randn(X.shape[1]);
h = np.dot(X,w)
loss(h, y_train)

(array([0.3575, 1.871 , 2.9722, 0.    , 0.4945, 0.    , 4.3644, 1.0606,
        3.4454, 3.0242, 0.997 , 1.9135, 0.8351, 5.2281, 5.0396, 4.4263,
        3.7704, 1.4742, 2.2681, 0.    , 1.0326, 0.    , 5.3443, 0.    ,
        0.1673, 2.9541, 3.9522, 0.    , 0.2244, 3.0803, 1.    , 1.    ,
        3.2127, 5.7837, 4.9177, 2.7745, 3.9306, 3.2592, 1.6041, 2.5888,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ]),
 array([-1., -1., -1.,  0., -1.,  0., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1.,  0., -1.,  0., -1.,  0., -1., -1.,
        -1.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [92]:
def reg(w, lbda):
    r = (lbda/2)*w.T*w
    g = lbda*w
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [120]:
def predict(w, X):
    preds = np.sign(np.dot(w,X.T))
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [107]:
y_train_old = y_train
np.place(y_train, y_train==0, [-1])

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [118]:
w_train = learn_reg_ERM(X_train,y_train,5)

loss: 1.8038747534866482
loss: 12.952532195558657
loss: 1.3685643672326038
loss: 3.5077238219025317
loss: 1.3584486173309334
loss: 0.7612972161270404
loss: 1.6897465778648795
loss: 0.7137139707255965
loss: 0.6978681183024692
loss: 0.7263799952058628
loss: 0.7096573905973178
loss: 0.6970564490097101
loss: 0.69607331479379
loss: 0.6966269951413167
loss: 0.6940256039890219
loss: 0.6940930396272798
loss: 0.6936023888726045
loss: 0.6937946050921046
loss: 0.6935755558601848
loss: 0.6937314995511821
loss: 0.6941088501326419
loss: 0.6934265539415296
loss: 0.693320680761117
loss: 0.693289356184031
loss: 0.693188204721665
loss: 0.7149999999999993
loss: 0.7023663360026242
loss: 0.697676583474694
loss: 0.6923125571751847
loss: 0.6912246600235882
loss: 0.6915614001942637
loss: 0.6917446174619812
loss: 0.6915984976324687
loss: 0.6916411149604411
loss: 0.6914438482531959
loss: 0.6914297352165372
loss: 0.6915077025007357
loss: 0.6915893894589399
loss: 0.6914883983740181
loss: 0.6915214146197922
loss: 

In [126]:
pred_train = predict(w_train, X_train)
zero_one_loss(y_train, pred_train)

0.38749999999999996

In [127]:
pred_test  = predict(w_train, X_test)
zero_one_loss(y_test, pred_test)

0.12299465240641716

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [129]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
forest_pred_train = clf.predict(X_train)
forest_pred_test  = clf.predict(X_test)
display(zero_one_loss(y_train, forest_pred_train))
display(zero_one_loss(y_test, forest_pred_test))

0.0625

0.25668449197860965

In [133]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
forest_pred_train = clf.predict(X_train)
forest_pred_test  = clf.predict(X_test)
display(zero_one_loss(y_train, forest_pred_train))
display(zero_one_loss(y_test, forest_pred_test))

0.0625

0.3048128342245989

# Conclusion
Apparently the ERM algorithm performs worse on training data, but is way better at predictions based on new data.